In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM track LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [4]:
%%sql 
WITH usa_tracks AS(
SELECT g.name, COUNT(t.track_id) track_amount, COUNT(il.invoice_line_id) amount_invoice 
FROM genre g
INNER JOIN track t USING (genre_id)
INNER JOIN invoice_line il USING(track_id)
INNER JOIN invoice i USING (invoice_id)
INNER JOIN customer c USING (customer_id)
WHERE country ="USA"
GROUP BY g.name)

SELECT name, track_amount, ROUND(CAST(track_amount AS float)/(SELECT SUM(amount_invoice) FROM usa_tracks)*100, 2) as per_track FROM usa_tracks
GROUP BY name
ORDER BY track_amount DESC;

 * sqlite:///chinook.db
Done.


name,track_amount,per_track
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on genres sold in USA the best artist to buy from the four options given are:
Red Tone - Punk, Meteor and the Girls - Pop, Slim Jim Bites - BLues

In [5]:
%%sql
SELECT e.first_name || " " || e.last_name employeeName, 
ROUND(SUM(i.total),2) totalPrice, e.hire_date
FROM invoice i 
INNER JOIN customer c  USING(customer_id)
INNER JOIN employee e ON c.support_rep_id = e.employee_id
GROUP BY employeeName;


 * sqlite:///chinook.db
Done.


employeeName,totalPrice,hire_date
Jane Peacock,1731.51,2017-04-01 00:00:00
Margaret Park,1584.0,2017-05-03 00:00:00
Steve Johnson,1393.92,2017-10-17 00:00:00


Here we can see that Jane Peacock made the most sales.  If you analyze from when they were hired it seems like Steve Johnson made more sales.  This is hard to compare to see who is better without knowing the date that this data was extracted.  We would be able to take that date and see sales per a month.  

In [6]:
%%sql
WITH other AS (
SELECT customer_id, country, COUNT(*) amount_customer,
    CASE 
        WHEN COUNT(*) = 1 THEN "Other"
        ELSE country
    END AS country_count
FROM customer
GROUP BY country
)

SELECT  o.country_count country, o.amount_customer,
SUM(il.unit_price) total_sales,   
SUM(il.unit_price)/COUNT(DISTINCT c.customer_id) avg_sales_per_cust, 
SUM(il.unit_price)/COUNT(DISTINCT i.invoice_id) avg_order
FROM invoice_line il
INNER JOIN invoice i USING (invoice_id)
INNER JOIN customer c USING (customer_id)
INNER JOIN other o USING (country)
GROUP BY o.country_count
ORDER BY o.amount_customer DESC;


 * sqlite:///chinook.db
Done.


country,amount_customer,total_sales,avg_sales_per_cust,avg_order
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
Other,1,1094.9400000000085,72.99600000000056,7.448571428571486


From here we can see that although USA has the most sales.  The highest avearage sales per a customer and average orders is Czech Republic, India, and United Kingdom.  The amount of customers in Czech Republic, India, and United Kingdom are very low so this infomation needs to be used with caution.

In [7]:
%%sql
WITH track_album AS (SELECT COUNT(track_id) track_album, album_id FROM track 
GROUP BY album_id),

purchased_track AS (SELECT t.album_id, COUNT(t.track_id) track_invoice, i.invoice_id 
FROM invoice i 
INNER JOIN invoice_line il USING (invoice_id)
INNER JOIN track t USING (track_id)
GROUP BY il.invoice_id, t.album_id
ORDER BY il.invoice_id),

purchased_track_album AS(SELECT ta.album_id, ta.track_album, MAX(pt.track_invoice) max_track, pt.invoice_id  
FROM track_album ta
INNER JOIN purchased_track pt USING (album_id)
GROUP BY pt.invoice_id
ORDER BY pt.invoice_id)

SELECT 
CASE WHEN max_track = track_album THEN "yes"
ELSE "no"
END purchased_album,
count(invoice_id) number_invoices, 
ROUND(CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice)*100, 2) percent_invoices
FROM purchased_track_album
GROUP BY purchased_album

 * sqlite:///chinook.db
Done.


purchased_album,number_invoices,percent_invoices
no,498,81.11
yes,116,18.89


There is a much higher percentage of buyers who do not buy whole albumns.  There is still almost 19% of buyers buying albums, which would seem to be a high enough percentage of sells to continue selling albums.

In [8]:
%%sql
WITH amount_track_album AS (SELECT COUNT(track_id) track_amount, album_id FROM track
GROUP BY album_id),

track_purchased_by_album AS (SELECT t.album_id,
COUNT(t.track_id) track_purchase_album, i.invoice_id
FROM invoice i
INNER JOIN invoice_line il USING (invoice_id)
INNER JOIN track t USING (track_id)
GROUP BY i.invoice_id),

album_purchase AS (SELECT tpa.album_id, ata.track_amount, tpa.track_purchase_album, tpa.invoice_id,
CASE
    WHEN track_amount = track_purchase_album THEN "yes"
    ELSE "no"
    END purchase_album
FROM track_purchased_by_album tpa
INNER JOIN amount_track_album ata USING(album_id))

SELECT COUNT(invoice_id), purchase_album FROM album_purchase
GROUP BY 2;

 * sqlite:///chinook.db
Done.


COUNT(invoice_id),purchase_album
492,no
122,yes
